# pyGSTi
Welcome! This notebook serves as a top-level "readme" document for pyGSTi.  It and the other notebooks in this directory describe and demonstrate how you can use the `pygsti` package to accomplish various tasks related to quantum characterization.  We'll start with a brief general introduction and then move on to specifics.

## What does pyGSTi do?
PyGSTi is a **Python framework for modeling and analyzing collections of qubits**, more affectionately called quantum information processors (QIPs).  We use "**QIP**" generously to include systems of just 1 or 2 qubits in addition to larger systems.  Initially pyGSTi was developed to perform a single type of data analysis called Gate Set Tomography (GST), from where pyGSTi derives it's name.  While GST is still a central capability of the package, pyGSTi is capable of modeling QIP behavior and analyzing QIP data much more generally.

For example, some things you might use pyGSTi for are:

- constructing and manipulating quantum circuits.
- constructing a model for a QIP.
- computing the outcome probabilities of a circuit predicted by a QIP model.
- simulating observed data based on a QIP model. 
- testing how a particular QIP model agrees with real data taken.
- running high-level quantum characterization protocols such as:
    - Gate Set Tomography (GST) on 1 or 2 qubits
    - Clifford Randomized Benchmarking (RB)
    - Direct Randomized Benchmarking (DRB)
    - Robust Phase Estimation (RPE)
    - Multi-qubit reduced-model tomography
- performing "drift-detection" tests on multiple passes of ideally identical data.
- computing the process fidelity or diamond distance between two gate matrices.
- making sweet-looking figures that compare QIP models and data.
- creating integrated reports which explain characterization results (especially GST).


## Getting Started
This documentation facilitates **two basic approaches** to getting started with pyGSTi.  

The first approach is a **top-down** approach that shows how to use pyGSTi as a stand-alone QCVV tool that runs protocols on data.  An overview of what protocols pyGSTi can run and how to invoke them is given in the [pyGSTi protocols overview](Tutorials/00-Protocols.ipynb) notebook.

The second is a **bottom-up** approach that starts by introducing the main components of pyGSTi and how they work together, and then explains what you can do with them.  If you like this approach or are ambivalent, you should read through these two notebooks to get a high-level overview of what pyGSTi can do:
- [Part1: pyGSTi's essential objects](Tutorials/01-Essential-Objects.ipynb)
- [Part2: Using these objects](Tutorials/02-Using-Essential-Objects.ipynb)

(These notebooks contain links to other notebooks covering more advanced functionality that you are free to explore as desired.)

If you have a particular problem you need solved and you want to know what 5 lines of Python code will get it done for you, then you should visit the [FAQ](FAQ.ipynb) and with links to associated example notebooks.  If your question isn't answered there, you should email us at pygsti@sandia.gov or add an issue to the pyGSTi github page.

In [5]:
from __future__ import print_function
import sys
from pygsti.processors import QubitProcessorSpec as QPS
from pygsti.processors import CliffordCompilationRules as CCR
from qiskit_ibm_runtime import QiskitRuntimeService
from pygsti.protocols import CliffordRBDesign
from qiskit import QuantumCircuit
import numpy as np
#ValueError: Exceeds the limit (4300) for integer string conversion 에러 해결
sys.set_int_max_str_digits(20000)

In [ ]:
#ibm brussels
backend = service.backend('ibm_brussels')

# 디바이스 사양 가져오기
n_qubits = backend.configuration().n_qubits
coupling_map = backend.configuration().coupling_map


#비표준 게이트 유니터리 정의
ECR_unitary = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1]
])
SX_unitary = np.array([
    [0.5 + 0.5j, 0.5 - 0.5j],
    [0.5 - 0.5j, 0.5 + 0.5j]
])
I_unitary = np.array([
    [1,0],
    [0,1]
])
X_unitary = np.array([
    [0,1],
    [1,0]
])
theta = np.pi / 2
RZ_unitary = np.array([
    [np.exp(-1j * theta / 2), 0],
    [0, np.exp(1j * theta / 2)]
])

nonstd_gate_unitaries = {
    'ECR': ECR_unitary,
    'sx': SX_unitary,
    'rz' : RZ_unitary,
    'x' : X_unitary,
    'id' : I_unitary
}

#프로세서 사양 정의
# 큐빗 서브셋 정의
n_qubits = 4
qubit_labels = [f'Q{i}' for i in range(n_qubits)]
gate_names = ['ECR', 'id', 'rz', 'sx', 'x']
availability = {'ECR': [('Q'+str(q[0]), 'Q'+str(q[1])) for q in coupling_map]}

pspec = QPS(
    num_qubits=n_qubits,
    gate_names=gate_names,
    qubit_labels=qubit_labels,
    nonstd_gate_unitaries=nonstd_gate_unitaries,
    availability=availability
)


#CliffordRBDesign 생성
#회로의 깊이와 시퀀스는 아래와 같이 설정하고 첫번째 큐빗에 대해서 RB를 적용할거임(1Q RB)
depths = [10, 20,30]
circuits_per_depth = 30
qubits=['Q0']
compilation_rules = CCR.create_standard(pspec, 'absolute', ('paulis', '1Qcliffords'))
compilations = {'absolute': compilation_rules}
rb_design = CliffordRBDesign(pspec, compilations, depths, circuits_per_depth, qubit_labels=qubits,num_processes=1)

#pyGSTi에서 RB구현에 사용되는 회로 확인
#Circuit 29 uses gates: {'rz', 'PH', 'x', 'H', 'HP', 'HPH', 'P'}
for depth_idx, circuits_at_depth in enumerate(rb_design.circuit_lists):
    print(f"Depth {rb_design.depths[depth_idx]}:")
    for circ_idx, circ in enumerate(circuits_at_depth):
        gate_names = set(op.name for op in circ)
        print(f"Circuit {circ_idx} uses gates: {gate_names}")

#ibm 머신이 알아먹도록 COMPOUND,HP 등을 매핑시켜줘야함
gate_name_mapping = {
    'rz': [('rz',np.pi/2)],    
    'x': 'x',      
    'sx': 'sx',     
    'P': [('rz',np.pi/2)], 
    'H': [('rz', np.pi), ('sx',), ('rz', np.pi)], 
    'HP': [('rz', np.pi), ('sx',), ('rz', np.pi/2)],
    'PH': [('rz', np.pi/2), ('rz', np.pi), ('sx',), ('rz', np.pi)],  
    'HPH': [('rz', np.pi), ('sx',), ('rz', np.pi/2), ('sx',), ('rz', np.pi)] 
}

def convert_pygsti_to_qiskit(circuit, n_qubits):
    qc = QuantumCircuit(n_qubits)
    for op in circuit:
        gate_name, args = op.name, op.args
        # 큐빗 인덱스는 항상 0으로 설정
        qubit_index = 0

        if gate_name in gate_name_mapping:
            mapping = gate_name_mapping[gate_name]
            if isinstance(mapping, str):  # 단일 게이트
                getattr(qc, mapping)(qubit_index)
            elif isinstance(mapping, list):  # 복합 게이트
                for m in mapping:
                    if len(m) == 1:  # 큐빗 인덱스만 필요
                        getattr(qc, m[0])(qubit_index)
                    elif len(m) == 2:  # 파라미터와 큐빗 인덱스가 모두 필요
                        getattr(qc, m[0])(m[1], qubit_index)  # m[1] = 파라미터
        else:
            raise ValueError(f"Unsupported gate: {gate_name}")
    return qc

# 변환된 Qiskit 회로 생성
qiskit_circuits = []
for depth_idx, circuits_at_depth in enumerate(rb_design.circuit_lists):
    for circ_idx, circ in enumerate(circuits_at_depth):
        qiskit_circuits.append(convert_pygsti_to_qiskit(circ, n_qubits))


# 실험 실행
job = backend.run(qiskit_circuits, shots=1024)
print(f"Job ID: {job.job_id()}")

# 결과 가져오기
result = job.result()
print("Experiment completed. Results:")
for idx, counts in enumerate(result.get_counts()):
    print(f"Circuit {idx} Results: {counts}")

- Creating a circuit to implement I  on qubit Q0...
- Generating a template for a compilation of I...
  - Checking all length 1 1-qubit circuits... (4)
Compilation template created!
- Creating a circuit to implement X  on qubit Q0...
- Generating a template for a compilation of X...
  - Checking all length 1 1-qubit circuits... (4)
Compilation template created!
- Creating a circuit to implement Y  on qubit Q0...
- Generating a template for a compilation of Y...
  - Checking all length 1 1-qubit circuits... (4)
  - Checking all length 2 1-qubit circuits... (16)
  - Checking all length 3 1-qubit circuits... (64)
Compilation template created!
- Creating a circuit to implement Z  on qubit Q0...
- Generating a template for a compilation of Z...
  - Checking all length 1 1-qubit circuits... (4)
  - Checking all length 2 1-qubit circuits... (16)
Compilation template created!
- Creating a circuit to implement C0  on qubit Q0...
- Generating a template for a compilation of C0...
  - Checking al

C:\Users\wjdrm\AppData\Local\Temp\ipykernel_14636\752924918.py:117: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = backend.run(qiskit_circuits, shots=1024)


Job ID: cwypq3c2ac5g008jrhpg


KeyboardInterrupt: 